## Optimizer

| Aspekt      | Erklärung                                                                 |
| ----------- | ------------------------------------------------------------------------- |
| **Was**     | Eine eigene Optimizer-Klasse verwaltet, wie Parameter aktualisiert werden |
| **Warum**   | Volle Kontrolle über Lernregeln (Forschung, neue Verfahren)               |
| **Wie**     | Erben von `torch.optim.Optimizer`, `__init__` + `step()` implementieren   |
| **Wichtig** | Immer `@torch.no_grad()` bei Updates verwenden, um Autograd zu vermeiden  |


In [1]:
import torch
from torch.optim import Optimizer

class MyOptimizer(Optimizer):
    def __init__(self, params, lr=0.01):
        # params = model.parameters()
        # defaults = Dictionary mit Hyperparametern
        defaults = dict(lr=lr)
        super().__init__(params, defaults)

    @torch.no_grad()  # verhindert das Aufzeichnen im Autograd-Graph
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        # Iteriere über alle Parametergruppen (z. B. für verschiedene LR)
        for group in self.param_groups:
            lr = group['lr']
            for p in group['params']:
                if p.grad is None:
                    continue  # Parameter ohne Gradient überspringen

                # Parameterupdate: θ ← θ - lr * ∂L/∂θ
                # p=p+(−lr)×p.grad
                p.add_(p.grad, alpha=-lr)

                #add_() is the in-place version of add() (notice the underscore _).
                # It means: modify the tensor itself rather than creating a new one.

        return loss


In [2]:
import torch.nn as nn

# Beispielmodell
model = nn.Linear(10, 1)
criterion = nn.MSELoss()

# Dein eigener Optimizer
optimizer = MyOptimizer(model.parameters(), lr=0.1)

# Trainingsloop
for epoch in range(5):
    inputs = torch.randn(8, 10)
    targets = torch.randn(8, 1)

    optimizer.zero_grad()           # Gradienten löschen
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()                 # Gradienten berechnen
    optimizer.step()                # Parameter updaten

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 1.2072
Epoch 2, Loss: 1.6639
Epoch 3, Loss: 1.5517
Epoch 4, Loss: 1.2095
Epoch 5, Loss: 1.1861
